# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Sst, Male, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISl sessions. The Female SST VISp sessions have 0 low-contamination units. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."
758798717,2019-10-03T00:00:00Z,735109609,brain_observatory_1.1,102.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,593,1486,4,"[MGv, MGd, TH, DG, CA1, VISal, nan, SGN, LP, S..."
762602078,2019-10-03T00:00:00Z,744915204,brain_observatory_1.1,110.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,531,2233,6,"[MB, APN, NOT, DG, CA1, VISam, nan, VISrl, LGv..."


In [7]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 884


,snr,presence_ratio,waveform_spread,cumulative_drift,isolation_distance,waveform_amplitude,firing_rate,nn_hit_rate,isi_violations,waveform_velocity_below,L_ratio,nn_miss_rate,waveform_halfwidth,waveform_repolarization_slope,amplitude_cutoff,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,waveform_duration,local_index_unit,max_drift,waveform_velocity_above,cluster_id,peak_channel_id,d_prime,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950910352,2.816993,0.99,60.0,419.57,69.455405,106.785900,4.532385,0.935531,0.029797,0.000000,0.002020,0.008277,0.096147,0.365539,0.057700,0.476196,0.080869,-0.153573,0.151089,6,34.38,-0.137353,6,850264156,4.576155,NaN,200.0,8.573516,NaN,3.563571,3.013333,3.472371,0.506292,0.121946,0.092779,0.109856,25.000,1.312811,0.479,0.32,4.0,0.987385,NaN,0.970335,1.000000,0.706667,180.0,120.0,0.000020,NaN,3.094523e-07,0.010614,3.288081e-03,50.000,4908,0.50,3.679539,0.0,5.399795,2.173257,4.740704,NaN,0.0185,False,False,False,False,False,0.074966,False,0.124258,NaN,0.053529,0.244172,0.055879,60,11,810755797,5,215.0,APN,8157.0,3521.0,6697.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910364,2.592082,0.99,60.0,231.42,102.847616,85.808775,31.486161,0.995333,0.005998,0.343384,0.000146,0.002786,0.274707,0.147357,0.065649,0.953652,0.153496,-0.010004,0.480737,7,23.43,-0.618090,7,850264158,5.602703,NaN,800.0,0.696467,NaN,1.127965,1.382684,1.474619,0.553213,0.037434,0.029263,0.032803,51.250,2.509759,0.979,0.16,4.0,0.148090,NaN,-0.017989,0.172414,0.015326,225.0,120.0,0.001521,NaN,8.555631e-01,0.236355,9.013584e-01,-18.750,4951,0.50,26.488752,0.0,31.565193,27.097733,31.374439,NaN,0.1475,False,False,False,False,False,0.030797,False,0.013569,NaN,0.003357,0.024370,0.004924,80,59,810755797,6,215.0,APN,8154.0,3513.0,6698.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910371,2.095077,0.99,70.0,209.31,76.907610,76.898055,17.328610,0.993333,0.005242,0.137353,0.004522,0.007975,0.164824,0.299231,0.015509,0.615593,0.089229,-0.071484,0.288442,8,57.44,0.274707,8,850264164,5.061817,NaN,300.0,6.767485,NaN,1.235328,3.876923,1.301639,0.223564,0.048717,0.050593,0.018864,86.667,1.024316,0.497,0.04,4.0,0.213891,NaN,0.204641,0.511111,-0.015873,225.0,30.0,0.329712,NaN,9.284294e-02,0.161964,9.037874e-01,46.667,4942,0.75,14.373947,0.0,17.881994,12.173311,19.064063,NaN,0.0435,False,False,False,False,False,0.005763,False,0.022789,NaN,0.005609,0.050257,0.005125,100,11,810755797,9,215.0,APN,8146.0,3487.0,6701.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910392,3.024744,0.99,70.0,179.71,65.671206,120.519555,16.262414,0.944000,0.096884,-0.274707,0.007306,0.002874,0.109883,0.497249,0.025891,0.616317,0.139601,-0.116365,0.206030,11,33.65,0.000000,11,850264172,4.219074,NaN,100.0,14.891282,NaN,6.331037,4.973913,5.240884,0.286195,0.089741,0.038626,0.066273,10.000,0.915356,0.942,0.08,4.0,1.000000,NaN,0.937998,0.760504,0.791262,180.0,150.0,0.000002,NaN,2.323203e-11,0.000867

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 41


,snr,presence_ratio,waveform_spread,cumulative_drift,isolation_distance,waveform_amplitude,firing_rate,nn_hit_rate,isi_violations,waveform_velocity_below,L_ratio,nn_miss_rate,waveform_halfwidth,waveform_repolarization_slope,amplitude_cutoff,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,waveform_duration,local_index_unit,max_drift,waveform_velocity_above,cluster_id,peak_channel_id,d_prime,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950930145,2.590404,0.98,30.0,202.02,78.408045,102.55557,9.910598,0.976667,0.011634,NaN,0.002632,0.001159,0.123618,0.237697,0.082525,7.031742,0.115805,-0.022541,0.590620,398,40.26,-5.837521,410,850261708,4.343827,NaN,0.0,23.322944,0.933333,4.232207,0.800000,3.128807,0.637168,0.114884,0.139538,0.135075,NaN,1.314005,0.4,0.02,1.0,-0.536232,-0.081967,0.621630,-1.000000,0.335562,90.0,150.0,0.379479,0.941158,0.000008,0.081553,0.022972,NaN,4998,0.50,6.251020,0.266445,15.236840,0.061404,15.854080,NaN,0.0345,False,False,False,False,False,0.145983,False,0.079757,0.076923,0.098016,0.824151,0.059138,2580,11,810755801,257,385.0,VISp,8586.0,1465.0,8212.0,probeC,See electrode locations,29999.98547,1249.999395,True
950930105,1.294513,0.99,40.0,145.08,100.175018,79.86069,11.054619,0.974667,0.011082,-0.686767,0.006947,0.007184,0.082412,0.200195,0.095761,0.601677,-0.023674,-0.077013,0.219765,396,37.75,3.777219,408,850261706,5.052111,NaN,800.0,16.188582,1.964928,4.818462,3.064912,4.698027,0.653696,0.265180,0.140406,0.161520,81.250,5.567877,0.0,0.02,1.0,0.610582,0.428571,0.523274,0.645320,0.626227,180.0,150.0,0.010041,0.022354,0.004629,0.019978,0.000445,-21.250,4930,0.50,7.612455,10.258134,11.119992,7.675480,11.051428,NaN,0.0855,False,False,False,False,False,0.279898,False,0.160895,0.316715,0.118449,0.144858,0.117219,2580,43,810755801,256,385.0,VISp,8584.0,1477.0,8213.0,probeC,See electrode locations,29999.98547,1249.999395,True
950930276,2.090243,0.99,60.0,101.96,75.644463,82.90425,8.772900,0.976667,0.025294,-0.343384,0.002472,0.002583,0.219765,0.195660,0.013349,0.517356,0.099918,-0.027413,0.865327,403,40.34,1.442211,415,850261712,3.670607,NaN,900.0,6.994679,0.272094,1.486415,0.146448,0.971354,0.266723,0.031243,0.314266,0.294280,46.667,1.128042,0.0,0.04,1.0,0.147541,0.228423,0.188034,-0.092437,0.045455,225.0,60.0,0.467916,0.093494,0.223447,0.371282,0.737922,-11.111,4952,0.50,8.939894,4.609499,9.112281,8.535133,8.715381,NaN,0.1175,False,False,False,False,False,0.357932,False,0.214877,0.171605,0.042784,0.090410,0.149341,2600,27,810755801,259,385.0,VISp,8591.0,1441.0,8210.0,probeC,See electrode locations,29999.98547,1249.999395,True
950933698,2.942125,0.98,50.0,152.21,47.931608,107.32878,1.277709,0.670732,0.025923,-2.747069,0.004305,0.002554,0.274707,0.277304,0.000023,0.476703,0.047764,-0.031722,0.824121,582,42.88,0.961474,601,850261710,4.428315,NaN,900.0,3.666667,1.066667,1.082857,0.819048,0.945175,0.481326,0.033753,0.392013,0.210443,51.111,0.815641,0.0,0.02,1.0,-0.333333,-0.737705,-1.000000,

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
950930145    15.854080
950930105    11.051428
950930276     8.715381
950933698     1.566024
950930237     3.918724
950930340     3.378509
950930423     3.709566
950930407    54.068134
950930392     1.674600
950930375     6.514554
950933732     2.114899
950930795     9.906385
950930888     3.405820
950930985    49.360070
950934181     0.498250
950930964    20.526840
950931043     3.600990
950931164     1.775183
950931272     0.562863
950931254    59.528902
950931423     5.663932
950931363     9.604638
950933840     5.542700
950931458    19.791455
950931315    18.294040
950931533    11.716872
950931517     6.204147
950931656    15.628935
950931805     2.148204
950931617     5.418803
950931581     2.297413
950931751     3.489749
950931727     9.699225
950931853     9.157678
950931899    19.796784
950931878     1.748538
950933890     2.670434
950932087    12.161833
950932032     8.799311
950932563    12.912539
950932696     0.583512
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

10.8551693427075


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
950930145     6.251020
950930105     7.612455
950930276     8.939894
950933698     1.778949
950930237     4.561311
950930340     4.332369
950930423     0.598649
950930407    50.409742
950930392     3.580738
950930375     4.524128
950933732     1.228638
950930795    12.027690
950930888     3.306645
950930985    32.967652
950934181     0.575277
950930964    17.081413
950931043     6.715279
950931164     0.863712
950931272     0.362801
950931254    50.222763
950931423     5.208298
950931363    11.337677
950933840     4.801408
950931458    12.496729
950931315    11.405138
950931533     2.839200
950931517     8.111241
950931656    13.138404
950931805     1.637653
950931617     1.877218
950931581     2.628849
950931751     2.779707
950931727    18.609641
950931853     6.824172
950931899    13.169213
950931878     1.291318
950933890     1.040597
950932087     7.935418
950932032     4.904990
950932563     9.238422
950932696     0.247534
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

8.76741347857022


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])